In [1]:
import os
import pandas as pd

path = os.getcwd()

train_data = pd.read_csv("../../datasets/train.csv", encoding = 'utf-8')

train_data.dropna(inplace = True)
train_data.drop(['감정_소분류','시스템문장1','시스템문장2','시스템문장3', '사람문장2', '사람문장3', '신체질환'], axis=1, inplace=True)
train_data.rename({'사람문장1':'text', '감정_대분류':'label'}, axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_data['label'] = label_encoder.fit_transform(train_data['label'])

train_data

,Unnamed: 0,연령,성별,상황키워드,label,text
304,305,중년,여성,"재정,은퇴,노후준비",4,지금까지 힘들게 일했는데 은퇴해서 돈이 없다고 하니 자식이 화를 내서 상처를 받았어.
305,306,중년,여성,"재정,은퇴,노후준비",4,친구한테 은퇴할 거라고 얘기했더니 앞으로 뭘 먹고 살 거냐면서 비웃더라고. 기분이 ...
306,307,중년,여성,"재정,은퇴,노후준비",4,친구한테 은퇴한다고 했더니 그게 말이나 되는 거냐며 날 한심한 사람 취급해서 서운했어.
307,308,중년,여성,"재정,은퇴,노후준비",4,그동안 열심히 달려와서 좀 쉬려고 하는데 은퇴한다고 하니 주변에서 다 말려서 기분이...
308,309,중년,여성,"재정,은퇴,노후준비",4,많은 고민 후 은퇴를 결심했는데 주변에서 다들 섣부른 생각이라고 해서 마음이 안 좋아.
...,...,...,...,...,...,...
51625,51626,노년,남성,재정,2,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.
51626,51627,노년,여성,재정,3,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.
51627,51628,노년,여성,재정,4,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.
51628,51629,노년,여성,대인관계,3,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.


In [2]:
id_to_label = {k: v for k, v in enumerate(label_encoder.classes_)}
id_to_label

{0: '기쁨', 1: '당황', 2: '분노', 3: '불안', 4: '상처', 5: '슬픔'}

In [3]:
from datasets import Dataset
from dataclasses import dataclass, field
from typing import Optional
import torch
from peft import LoraConfig
from tqdm import tqdm
import pandas as pd
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer

question_template = "### Human: 다음 문장의 감정을 기쁨, 당황, 분노, 불안, 상처, 슬픔 중 하나로 분류해줘. "

train_instructions = [f'{question_template}\ntext: {x}\n\n### Assistant: {id_to_label[y]}' for x,y in zip(train_data['text'],train_data['label'])]
# validation_instructions = [f'{question_template}\ntext: {x}\n\n### Assistant: {id_to_label[y]}' for x,y in zip(validation_data['premise'],validation_data['hypothesis'],validation_data['label'])]

datasets = Dataset.from_dict({"text": train_instructions})
# ds_validation = Dataset.from_dict({"text": validation_instructions})
# instructions_ds_dict = DatasetDict({"train": ds_train, "eval": ds_validation})
datasets = datasets.train_test_split(test_size=0.1)

In [4]:
model_name = "beomi/open-llama-2-ko-7b"

@dataclass
class ScriptArguments:
    model_name: Optional[str] = field(default=model_name, metadata={"help": "the model name"})
    dataset_text_field: Optional[str] = field(default="text", metadata={"help": "the text field of the dataset"})
    log_with: Optional[str] = field(default=None, metadata={"help": "use 'wandb' to log with wandb"})
    learning_rate: Optional[float] = field(default=1.41e-5, metadata={"help": "the learning rate"})
    batch_size: Optional[int] = field(default=8, metadata={"help": "the batch size"})
    seq_length: Optional[int] = field(default=512, metadata={"help": "Input sequence length"})
    gradient_accumulation_steps: Optional[int] = field(
        default=2, metadata={"help": "the number of gradient accumulation steps"}
    )
    load_in_8bit: Optional[bool] = field(default=False, metadata={"help": "load the model in 8 bits precision"})
    load_in_4bit: Optional[bool] = field(default=True, metadata={"help": "load the model in 4 bits precision"})
    use_peft: Optional[bool] = field(default=True, metadata={"help": "Wether to use PEFT or not to train adapters"})
    trust_remote_code: Optional[bool] = field(default=True, metadata={"help": "Enable `trust_remote_code`"})
    output_dir: Optional[str] = field(default="output", metadata={"help": "the output directory"})
    peft_lora_r: Optional[int] = field(default=64, metadata={"help": "the r parameter of the LoRA adapters"})
    peft_lora_alpha: Optional[int] = field(default=16, metadata={"help": "the alpha parameter of the LoRA adapters"})
    logging_steps: Optional[int] = field(default=1, metadata={"help": "the number of logging steps"})
    use_auth_token: Optional[bool] = field(default=False, metadata={"help": "Use HF auth token to access the model"})
    num_train_epochs: Optional[int] = field(default=3, metadata={"help": "the number of training epochs"})
    max_steps: Optional[int] = field(default=-1, metadata={"help": "the number of training steps"})
    save_steps: Optional[int] = field(
        default=100, metadata={"help": "Number of updates steps before two checkpoint saves"}
    )
    save_total_limit: Optional[int] = field(default=10, metadata={"help": "Limits total number of checkpoints."})
    push_to_hub: Optional[bool] = field(default=False, metadata={"help": "Push the model to HF Hub"})
    hub_model_id: Optional[str] = field(default=None, metadata={"help": "The name of the model on HF Hub"})


script_args = ScriptArguments()

In [5]:
if script_args.load_in_8bit and script_args.load_in_4bit:
    raise ValueError("You can't load the model in 8 bits and 4 bits at the same time")
elif script_args.load_in_8bit or script_args.load_in_4bit:
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=script_args.load_in_8bit, load_in_4bit=script_args.load_in_4bit
    )
    device_map = {"": 0}
    torch_dtype = torch.bfloat16
else:
    device_map = None
    quantization_config = None
    torch_dtype = None

model = AutoModelForCausalLM.from_pretrained(
    script_args.model_name,
    quantization_config=quantization_config,
    device_map=device_map,
    trust_remote_code=script_args.trust_remote_code,
    torch_dtype=torch_dtype,
    use_auth_token=script_args.use_auth_token,
)

c:\Users\JH\miniconda3\envs\capstone\Lib\site-packages\transformers\models\auto\auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
dataset = datasets

training_args = TrainingArguments(
    output_dir=script_args.output_dir,
    per_device_train_batch_size=script_args.batch_size,
    gradient_accumulation_steps=script_args.gradient_accumulation_steps,
    learning_rate=script_args.learning_rate,
    logging_steps=script_args.logging_steps,
    num_train_epochs=script_args.num_train_epochs,
    max_steps=script_args.max_steps,
    report_to=script_args.log_with,
    save_steps=script_args.save_steps,
    save_total_limit=script_args.save_total_limit,
    push_to_hub=script_args.push_to_hub,
    hub_model_id=script_args.hub_model_id,
)

if script_args.use_peft:
    peft_config = LoraConfig(
        r=script_args.peft_lora_r,
        lora_alpha=script_args.peft_lora_alpha,
        bias="none",
        task_type="CAUSAL_LM",
    )
else:
    peft_config = None

trainer = SFTTrainer(
    model=model,
    args=training_args,
    max_seq_length=script_args.seq_length,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    dataset_text_field=script_args.dataset_text_field,
    peft_config=peft_config,
)

Map:   0%|          | 0/38424 [00:00<?, ? examples/s]

Map:   0%|          | 0/4270 [00:00<?, ? examples/s]

c:\Users\JH\miniconda3\envs\capstone\Lib\site-packages\trl\trainer\sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
c:\Users\JH\miniconda3\envs\capstone\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [7]:
trainer.train()

trainer.save_model(training_args.output_dir)

  0%|          | 0/7203 [00:00<?, ?it/s]

c:\Users\JH\miniconda3\envs\capstone\Lib\site-packages\transformers\models\llama\modeling_llama.py:670: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 4.2701, 'grad_norm': 0.37196606397628784, 'learning_rate': 1.4098042482299042e-05, 'epoch': 0.0}
{'loss': 4.3762, 'grad_norm': 0.378172367811203, 'learning_rate': 1.4096084964598085e-05, 'epoch': 0.0}
{'loss': 4.1956, 'grad_norm': 0.47833016514778137, 'learning_rate': 1.4094127446897126e-05, 'epoch': 0.0}
{'loss': 4.3382, 'grad_norm': 0.7295925617218018, 'learning_rate': 1.4092169929196169e-05, 'epoch': 0.0}
{'loss': 4.1718, 'grad_norm': 0.36602410674095154, 'learning_rate': 1.409021241149521e-05, 'epoch': 0.0}
{'loss': 4.3065, 'grad_norm': 0.3946073353290558, 'learning_rate': 1.4088254893794253e-05, 'epoch': 0.0}
{'loss': 4.4242, 'grad_norm': 0.42619261145591736, 'learning_rate': 1.4086297376093294e-05, 'epoch': 0.0}
{'loss': 4.2549, 'grad_norm': 0.46636444330215454, 'learning_rate': 1.4084339858392337e-05, 'epoch': 0.0}
{'loss': 4.3534, 'grad_norm': 0.5367198586463928, 'learning_rate': 1.4082382340691378e-05, 'epoch': 0.0}
{'loss': 4.2732, 'grad_norm': 0.42548972368240356, '

In [10]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(training_args.output_dir)

model.eval()
pipeline_ = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map={'':0},
)

In [11]:
queries = [dataset['test']['text'][i].split('### Assistant: ')[0] + '### Assistant:' for i in range(len(dataset['test']))]
sequences = pipeline_(
    queries,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=3,
    early_stopping=True,
    # do_sample=True,
)

c:\Users\JH\miniconda3\envs\capstone\Lib\site-packages\transformers\generation\configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [12]:
results = []

for seq in sequences:
    result = seq[0]['generated_text'].split('### Assistant:')[1]
    results.append(result)

labels = []

for label in dataset['test']['text']:
    result = label.split('### Assistant:')[1]
    labels.append(result)

print("Accuracy: ", (len([1 for x, y in zip(results, labels) if y in x]) / len(labels)))

Accuracy:  0.4569086651053864


In [33]:
queries[3]

'### Human: 다음 문장의 감정을 기쁨, 당황, 분노, 불안, 상처, 슬픔 중 하나로 분류해줘. \ntext: 노인정 사람들 중에 병원 갈 때마다 자식들이 데리러 오는 사람들이 있는데 조금 부럽다.\n\n### Assistant:'

In [32]:
sequences[3]

[{'generated_text': '### Human: 다음 문장의 감정을 기쁨, 당황, 분노, 불안, 상처, 슬픔 중 하나로 분류해줘. \ntext: 그냥 모든 게 다 짜증나.\n\n### Assistant: 분노.\n'}]

In [34]:
dataset['test']['text'][3]

'### Human: 다음 문장의 감정을 기쁨, 당황, 분노, 불안, 상처, 슬픔 중 하나로 분류해줘. \ntext: 노인정 사람들 중에 병원 갈 때마다 자식들이 데리러 오는 사람들이 있는데 조금 부럽다.\n\n### Assistant: 상처'